<a href="https://colab.research.google.com/github/canfielder/DSBA-6190_Proj4_Serverless-Pipeline/blob/master/notebooks/DSBA_6190_Project_4_Serverless_Data_Engineering_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

## Installs

### Boto3

In [0]:
!pip -q install boto3

## Packages

In [0]:
# General
import os

# AWS Connection
from google.colab import drive
import boto3



# Set Up AWS Connection

## Mount Google Drive


In [40]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


Now we verify we can access AWS Credentials. We will the AWS credentials from my personal Google Drive and into the appropriate local folder.

In [41]:
path = "/content/gdrive/My Drive/aws"

aws_dir = os.listdir(path)
print(aws_dir)

aws_credentials = aws_dir.pop(0)

['credentials', 'user_evan.canfield_accessKeys.csv']


Establish the source location of the AWS credential file.

In [42]:
aws_credentials_src = os.path.join(path, aws_credentials)
aws_credentials_src = aws_credentials_src.replace(" ", "\ ")
print(aws_credentials_src)

/content/gdrive/My\ Drive/aws/credentials


Establish the destination of where to copy the AWS credential file.

In [43]:
aws_credentials_dst = "~/.aws/"
print(aws_credentials_dst)    

~/.aws/


Copy the credentials from the my mounted Google Drive to the local folder.

In [0]:
#!/usr/bin/env python3
mkdir -p ~/.aws &&\
  cp -r {aws_credentials_src} {aws_credentials_dst} 

Verify the credentials were correctly copied.

In [45]:
#!/usr/bin/env python3
ls -R {aws_credentials_dst}

/root/.aws/:
credentials


## API Call Test
Test that we have access to the AWS function.

In [50]:
session = boto3.Session(profile_name='dsba_6190_proj_4')
comprehend = session.client(service_name='comprehend', region_name="us-east-1")
text_sample = "I hope I don't have to be quarentined."
comprehend.detect_sentiment(Text=text_sample, LanguageCode='en')

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '163',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Fri, 13 Mar 2020 19:22:12 GMT',
   'x-amzn-requestid': '12e706cf-5dba-47de-b3ca-6db53a0986c7'},
  'HTTPStatusCode': 200,
  'RequestId': '12e706cf-5dba-47de-b3ca-6db53a0986c7',
  'RetryAttempts': 0},
 'Sentiment': 'NEUTRAL',
 'SentimentScore': {'Mixed': 1.1915316463273484e-05,
  'Negative': 0.11930244415998459,
  'Neutral': 0.44884905219078064,
  'Positive': 0.4318366050720215}}